# Import

In [ ]:
import pandas as pd
from cyvcf2 import VCF

# Define Variables

In [ ]:
gene = 'MYBPC3'
variants = 'NCSS'

if gene == 'ABCA4':
    chromosome = 1
elif gene == 'MYBPC3':
    chromosome = 11

# Load the variants 

In [ ]:
# create a dataframe to store the variants
data = pd.DataFrame(columns=['location', 'ref', 'alt', 'score'])

locations = []

# load the variants and store them in de dataframe
for variant in VCF(('../SpliceAI/' + gene + '_' + variants + '_variants.vcf')): 
    data = data.append({'location': variant.POS, 'ref': variant.REF, 'alt': variant.ALT[0]}, ignore_index=True)
    locations.append(variant.POS)
    
print(data.head())

# Get the scores for the chromosome of interest

In [ ]:
scores = []

# open the file
# scap file can be downloaded from http://bejerano.stanford.edu/scap/
with open('scap_COMBINED_v1.0.vcf') as f:
    header = f.readline()
    for line in f:
        content = line.split('\t')
        # check if the variant is located on chromosome 11
        if content[0] ==  str(chromosome):
            # check if the variant position is included into the ABCA4 variant list
            if int(content[1]) in locations:
                scores.append(content)
        elif content[0] == str(chromosome + 1):
            break

# Get the score for each variant

In [ ]:
for prediction in scores:

    # get the position and alternative bases
    pos = int(prediction[1])
    alt = prediction[4]

    # split the score for the different alternative bases
    all_scores = (prediction[-1]).split(':')
    # and store them in a dictionary 
    score = {}
    score[alt[0]] = all_scores[2]
    score[alt[2]] = all_scores[9]
    score[alt[4]] = all_scores[16]

    # look the position and the corresponing base up in the dataframe
    i=data[data['location'] == pos]
    try:
        result = score[i.alt.item()]
    except:
        result = None

    # store the index to assign the score to the right row in the dataframe
    index = data.index[data['location'] == pos].tolist()[0]

    # add the score to the dataframe
    data.at[index,'score'] = result

# write the result to a file
data.to_excel(('scap_' + gene + '_' + variants + '.xlsx'))